In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pickle



# Read the CSV file (using a raw string for the Windows path)
df = pd.read_csv(r"C:\Users\Overkill\Desktop\retail_store_inventory.csv")

# Display the first few rows of the data
print(df.head())

         Date Store ID Product ID     Category Region  Inventory Level  \
0  2022-01-01     S001      P0001    Groceries  North              231   
1  2022-01-01     S001      P0002         Toys  South              204   
2  2022-01-01     S001      P0003         Toys   West              102   
3  2022-01-01     S001      P0004         Toys  North              469   
4  2022-01-01     S001      P0005  Electronics   East              166   

   Units Sold  Units Ordered  Demand Forecast  Price  Discount  \
0         127             55           135.47  33.50        20   
1         150             66           144.04  63.01        20   
2          65             51            74.02  27.99        10   
3          61            164            62.18  32.72        10   
4          14            135             9.26  73.64         0   

  Weather Condition  Holiday/Promotion  Competitor Pricing Seasonality  
0             Rainy                  0               29.69      Autumn  
1           

In [5]:
print("Columns:", df.columns.tolist())

Columns: ['Date', 'Store ID', 'Product ID', 'Category', 'Region', 'Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount', 'Weather Condition', 'Holiday/Promotion', 'Competitor Pricing', 'Seasonality']


In [9]:


# 1. Read the CSV file
df = pd.read_csv(r"C:\Users\Overkill\Desktop\retail_store_inventory.csv")

# Display initial columns and first few rows for inspection
print("Initial columns:", df.columns.tolist())
print(df.head())

# 2. Convert 'Date' to datetime and extract features
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])  # Drop rows where date conversion failed
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# 3. Define columns
# Remove "Store ID" and "Product ID" from numeric_cols because they are non-numeric.
numeric_cols = ['Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount', 'Competitor Pricing']
categorical_cols = ['Store ID', 'Product ID', 'Category', 'Region', 'Weather Condition', 'Holiday/Promotion', 'Seasonality']

# 4. Fill missing values
# For numeric columns, fill NaNs with the median value
for col in numeric_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)

# For categorical columns, fill NaNs with the mode (or a placeholder)
for col in categorical_cols:
    if col in df.columns:
        mode_val = df[col].mode()[0] if not df[col].mode().empty else 'Unknown'
        df[col].fillna(mode_val, inplace=True)

# Optionally, forward-fill any remaining missing values
df.fillna(method='ffill', inplace=True)

# 5. Encode categorical variables using one-hot encoding
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# 6. Drop columns not needed for modeling (if desired)
# For example, you might drop the original 'Date' column since you've extracted Year, Month, Day.
df_model = df_encoded.drop(columns=['Date'])

# Display final preprocessed data shape and preview
print("Final DataFrame shape:", df_model.shape)
print(df_model.head())


Initial columns: ['Date', 'Store ID', 'Product ID', 'Category', 'Region', 'Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount', 'Weather Condition', 'Holiday/Promotion', 'Competitor Pricing', 'Seasonality']
         Date Store ID Product ID     Category Region  Inventory Level  \
0  2022-01-01     S001      P0001    Groceries  North              231   
1  2022-01-01     S001      P0002         Toys  South              204   
2  2022-01-01     S001      P0003         Toys   West              102   
3  2022-01-01     S001      P0004         Toys  North              469   
4  2022-01-01     S001      P0005  Electronics   East              166   

   Units Sold  Units Ordered  Demand Forecast  Price  Discount  \
0         127             55           135.47  33.50        20   
1         150             66           144.04  63.01        20   
2          65             51            74.02  27.99        10   
3          61            164            62.18  3

C:\Users\Overkill\AppData\Local\Temp\ipykernel_6160\241342921.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_val, inplace=True)
C:\Users\Overkill\AppData\Local\Temp\ipykernel_6160\241342921.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

Final DataFrame shape: (73100, 47)
   Inventory Level  Units Sold  Units Ordered  Demand Forecast  Price  \
0              231         127             55           135.47  33.50   
1              204         150             66           144.04  63.01   
2              102          65             51            74.02  27.99   
3              469          61            164            62.18  32.72   
4              166          14            135             9.26  73.64   

   Discount  Competitor Pricing  Year  Month  Day  ...  Region_North  \
0        20               29.69  2022      1    1  ...          True   
1        20               66.16  2022      1    1  ...         False   
2        10               31.32  2022      1    1  ...         False   
3        10               34.74  2022      1    1  ...          True   
4         0               68.95  2022      1    1  ...         False   

   Region_South  Region_West  Weather Condition_Rainy  \
0         False        False        

In [11]:

df = pd.read_csv(r"C:\Users\Overkill\Desktop\retail_store_inventory.csv")
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# Define columns: adjust numeric_cols and categorical_cols as needed.
numeric_cols = ['Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount', 'Competitor Pricing']
categorical_cols = ['Store ID', 'Product ID', 'Category', 'Region', 'Weather Condition', 'Holiday/Promotion', 'Seasonality']

# Fill missing values for numeric columns with median and categorical with mode.
for col in numeric_cols:
    if col in df.columns:
        df[col].fillna(df[col].median(), inplace=True)

for col in categorical_cols:
    if col in df.columns:
        df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else 'Unknown', inplace=True)

# Forward-fill any remaining missing values.
df.fillna(method='ffill', inplace=True)

# One-hot encode categorical columns.
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Drop the original 'Date' column as we now have Year, Month, Day.
df_model = df_encoded.drop(columns=['Date'])

target = 'Units Sold'
if target not in df_model.columns:
    raise ValueError("Target column not found in the DataFrame.")

# X: All columns except the target.
X = df_model.drop(columns=[target])
y = df_model[target]

# Split the dataset into training and testing sets (80% train, 20% test).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a simple Linear Regression model.
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model on the test set.
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

# Save the trained model to a file for later use.
with open('forecast_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model training complete and saved as forecast_model.pkl")


C:\Users\Overkill\AppData\Local\Temp\ipykernel_6160\1201644626.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\Overkill\AppData\Local\Temp\ipykernel_6160\1201644626.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

Mean Squared Error: 74.0382755593537
R^2 Score: 0.9937465900856656
Model training complete and saved as forecast_model.pkl


In [13]:


df = pd.read_csv(r"C:\Users\Overkill\Desktop\retail_store_inventory.csv")

# Display initial columns and first few rows for inspection
print("Initial columns:", df.columns.tolist())
print(df.head())

# Reset index to preserve row order
df.reset_index(drop=True, inplace=True)

# Save reference columns for later display: Product ID and Units Sold
df_reference = df[['Product ID', 'Units Sold']].copy()

# 2. Convert 'Date' to datetime and extract features
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])  # Drop rows with invalid dates
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# 3. Define columns
# Numeric columns (only truly numeric ones)
numeric_cols = ['Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount', 'Competitor Pricing']
# For categorical columns, we exclude "Product ID" so we can keep it for reference.
categorical_cols = ['Store ID', 'Category', 'Region', 'Weather Condition', 'Holiday/Promotion', 'Seasonality']

# 4. Fill missing values
# For numeric columns, fill NaNs with the median
for col in numeric_cols:
    if col in df.columns:
        df[col].fillna(df[col].median(), inplace=True)
# For categorical columns, fill NaNs with the mode (or 'Unknown' if mode isn't available)
for col in categorical_cols:
    if col in df.columns:
        df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else 'Unknown', inplace=True)
# Optionally, forward-fill any remaining missing values
df.fillna(method='ffill', inplace=True)

# 5. One-hot encode categorical variables (excluding Product ID)
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# 6. Drop columns not needed for modeling
# We drop the original 'Date' since we've extracted Year, Month, and Day.
df_model = df_encoded.drop(columns=['Date'])

# Display final preprocessed DataFrame shape and a preview
print("Final DataFrame shape:", df_model.shape)
print(df_model.head())

# Feature Selection & Target Definition
# We use "Units Sold" as the target.
target = 'Units Sold'
if target not in df_model.columns:
    raise ValueError("Target column not found in the DataFrame.")

# Remove target and "Product ID" (if present) from features.
if 'Product ID' in df_model.columns:
    X = df_model.drop(columns=[target, 'Product ID'])
else:
    X = df_model.drop(columns=[target])
y = df_model[target]

# 7. Split the data into training and testing sets.
# Also return the original indices so we can reference the Product IDs later.
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(
    X, y, df.index, test_size=0.2, random_state=42
)

# 8. Train a Linear Regression model.
model = LinearRegression()
model.fit(X_train, y_train)

# 9. Make predictions on both training and testing sets.
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# 10. Calculate accuracy metrics.
train_mse = mean_squared_error(y_train, train_predictions)
train_r2 = r2_score(y_train, train_predictions)
test_mse = mean_squared_error(y_test, test_predictions)
test_r2 = r2_score(y_test, test_predictions)

print("\n--- Model Accuracy Metrics ---")
print("Training MSE:", train_mse)
print("Training R^2:", train_r2)
print("Testing MSE:", test_mse)
print("Testing R^2:", test_r2)

# 11. Save the trained model to a file.
with open('forecast_model.pkl', 'wb') as f:
    pickle.dump(model, f)
print("\nModel training complete and saved as forecast_model.pkl")

# 12. Display actual vs predicted for a particular product from the test set.
# Use the original reference DataFrame (df_reference) to get the Product IDs and actual Units Sold.
df_test_reference = df_reference.loc[indices_test].reset_index(drop=True)
test_results = pd.DataFrame({
    'Product ID': df_test_reference['Product ID'],
    'Actual Units Sold': y_test.reset_index(drop=True),
    'Predicted Units Sold': test_predictions
})

# For example, filter results for a particular Product ID (e.g., "P0001")
product_filter = 'P0001'
product_results = test_results[test_results['Product ID'] == product_filter]

print("\n--- Actual vs Predicted Units Sold for Product ID", product_filter, "---")
print(product_results)


Initial columns: ['Date', 'Store ID', 'Product ID', 'Category', 'Region', 'Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount', 'Weather Condition', 'Holiday/Promotion', 'Competitor Pricing', 'Seasonality']
         Date Store ID Product ID     Category Region  Inventory Level  \
0  2022-01-01     S001      P0001    Groceries  North              231   
1  2022-01-01     S001      P0002         Toys  South              204   
2  2022-01-01     S001      P0003         Toys   West              102   
3  2022-01-01     S001      P0004         Toys  North              469   
4  2022-01-01     S001      P0005  Electronics   East              166   

   Units Sold  Units Ordered  Demand Forecast  Price  Discount  \
0         127             55           135.47  33.50        20   
1         150             66           144.04  63.01        20   
2          65             51            74.02  27.99        10   
3          61            164            62.18  3

C:\Users\Overkill\AppData\Local\Temp\ipykernel_6160\1016202941.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\Overkill\AppData\Local\Temp\ipykernel_6160\1016202941.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

Final DataFrame shape: (73100, 29)
  Product ID  Inventory Level  Units Sold  Units Ordered  Demand Forecast  \
0      P0001              231         127             55           135.47   
1      P0002              204         150             66           144.04   
2      P0003              102          65             51            74.02   
3      P0004              469          61            164            62.18   
4      P0005              166          14            135             9.26   

   Price  Discount  Competitor Pricing  Year  Month  ...  Region_North  \
0  33.50        20               29.69  2022      1  ...          True   
1  63.01        20               66.16  2022      1  ...         False   
2  27.99        10               31.32  2022      1  ...         False   
3  32.72        10               34.74  2022      1  ...          True   
4  73.64         0               68.95  2022      1  ...         False   

   Region_South  Region_West  Weather Condition_Rainy  \


In [15]:

df = pd.read_csv(r"C:\Users\Overkill\Desktop\retail_store_inventory.csv")
print("Initial columns:", df.columns.tolist())
print(df.head())

# Reset index to preserve row order
df.reset_index(drop=True, inplace=True)

# Preserve reference columns for later display: Product ID, Category, and actual Units Sold
df_reference = df[['Product ID', 'Category', 'Units Sold']].copy()

# 2. Convert 'Date' to datetime and extract features
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])  # Drop rows with invalid dates
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# 3. Define columns
# Numeric columns (only truly numeric ones)
numeric_cols = ['Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount', 'Competitor Pricing']
# Categorical columns – we keep Category (for display) and others for encoding.
categorical_cols = ['Store ID', 'Category', 'Region', 'Weather Condition', 'Holiday/Promotion', 'Seasonality']

# 4. Fill missing values
# For numeric columns, fill NaNs with the median
for col in numeric_cols:
    if col in df.columns:
        df[col].fillna(df[col].median(), inplace=True)

# For categorical columns, fill NaNs with the mode
for col in categorical_cols:
    if col in df.columns:
        df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else 'Unknown', inplace=True)

# Optionally, forward-fill any remaining missing values
df.fillna(method='ffill', inplace=True)

# 5. One-hot encode categorical variables for modeling.
# We want to keep 'Category' for display, so we exclude it from encoding.
categorical_cols_for_encoding = [col for col in categorical_cols if col != 'Category']
df_encoded = pd.get_dummies(df, columns=categorical_cols_for_encoding, drop_first=True)

# 6. Drop columns not needed for modeling.
# Drop the original 'Date' since we've extracted Year, Month, and Day.
df_model = df_encoded.drop(columns=['Date'])

print("Final DataFrame shape:", df_model.shape)
print(df_model.head())

# 7. Feature Selection & Target Definition
# We use "Units Sold" as the target.
target = 'Units Sold'
if target not in df_model.columns:
    raise ValueError("Target column not found in the DataFrame.")

# For modeling, drop the target, Product ID, and Category (we keep Category for later reference)
features_to_drop = ['Units Sold', 'Product ID', 'Category']
X = df_model.drop(columns=features_to_drop)
y = df_model[target]

# 8. Split the data into training and testing sets.
# Also return the original indices to later reference Product ID and Category.
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(
    X, y, df.index, test_size=0.2, random_state=42
)

# 9. Train a Linear Regression model.
model = LinearRegression()
model.fit(X_train, y_train)

# 10. Make predictions on both training and testing sets.
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

# 11. Calculate overall accuracy metrics.
train_mse = mean_squared_error(y_train, train_preds)
train_r2 = r2_score(y_train, train_preds)
test_mse = mean_squared_error(y_test, test_preds)
test_r2 = r2_score(y_test, test_preds)
print("\n--- Overall Model Accuracy ---")
print("Training MSE:", train_mse, "Training R^2:", train_r2)
print("Testing MSE:", test_mse, "Testing R^2:", test_r2)

# 12. Prepare results for each product in the test set.
# Retrieve reference data for test indices.
df_test_reference = df_reference.loc[indices_test].reset_index(drop=True)
results = pd.DataFrame({
    'Product ID': df_test_reference['Product ID'],
    'Category': df_test_reference['Category'],
    'Actual Units Sold': y_test.reset_index(drop=True),
    'Predicted Units Sold': test_preds
})

# 13. Compute a simple accuracy metric for each record.
# Here, we define accuracy as:
#   If actual > 0: accuracy (%) = max(0, (1 - abs(actual - predicted)/actual)) * 100
#   If actual == 0: accuracy = 100 if predicted is also 0, else 0.
def calc_accuracy(actual, predicted):
    if actual == 0:
        return 100 if predicted == 0 else 0
    else:
        return max(0, (1 - abs(actual - predicted) / actual)) * 100

results['Accuracy (%)'] = results.apply(lambda row: calc_accuracy(row['Actual Units Sold'], row['Predicted Units Sold']), axis=1)

print("\n--- Prediction Results for All Products (Test Set) ---")
print(results)

# 14. Save the trained model to a .pkl file.
with open('forecast_model.pkl', 'wb') as f:
    pickle.dump(model, f)
print("\nModel training complete and saved as forecast_model.pkl")


Initial columns: ['Date', 'Store ID', 'Product ID', 'Category', 'Region', 'Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount', 'Weather Condition', 'Holiday/Promotion', 'Competitor Pricing', 'Seasonality']
         Date Store ID Product ID     Category Region  Inventory Level  \
0  2022-01-01     S001      P0001    Groceries  North              231   
1  2022-01-01     S001      P0002         Toys  South              204   
2  2022-01-01     S001      P0003         Toys   West              102   
3  2022-01-01     S001      P0004         Toys  North              469   
4  2022-01-01     S001      P0005  Electronics   East              166   

   Units Sold  Units Ordered  Demand Forecast  Price  Discount  \
0         127             55           135.47  33.50        20   
1         150             66           144.04  63.01        20   
2          65             51            74.02  27.99        10   
3          61            164            62.18  3

C:\Users\Overkill\AppData\Local\Temp\ipykernel_6160\2735092927.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\Overkill\AppData\Local\Temp\ipykernel_6160\2735092927.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

Final DataFrame shape: (73100, 26)
  Product ID     Category  Inventory Level  Units Sold  Units Ordered  \
0      P0001    Groceries              231         127             55   
1      P0002         Toys              204         150             66   
2      P0003         Toys              102          65             51   
3      P0004         Toys              469          61            164   
4      P0005  Electronics              166          14            135   

   Demand Forecast  Price  Discount  Competitor Pricing  Year  ...  \
0           135.47  33.50        20               29.69  2022  ...   
1           144.04  63.01        20               66.16  2022  ...   
2            74.02  27.99        10               31.32  2022  ...   
3            62.18  32.72        10               34.74  2022  ...   
4             9.26  73.64         0               68.95  2022  ...   

   Region_North  Region_South  Region_West  Weather Condition_Rainy  \
0          True         False     